# Spatial maps of WoFS fields

By: Ty Janoski

In this notebook, we will focus on making spatial maps of relevant fields from
WoFS. Other types of plots will be elsewhere.


## Import statements

In [1]:
import xarray as xr

## Read in data

In [2]:
def load_data(init: str, keyword: str) -> xr.Dataset:

    # Note that this function depends on a consistent file structure so if you
    # change that, you'll need to change this function.
    # First, determine starting date
    start_date = "2021-09-02" if init in ["00Z", "01Z"] else "2021-09-01"
    new_dates = xr.date_range(
        start=start_date + " " + init[:-1] + ":00:00",
        end="2021-09-02 " + str((int(init[:-1]) + 6) % 24).zfill(2) + ":00:00",
        freq="5min",
        use_cftime=True,
    )

    # read in data
    data = xr.open_mfdataset(
        "/mnt/drive2/wof-runs/post-processed/"
        + init
        + "/wofs_"
        + keyword.upper()
        + "*.nc",
        concat_dim="time",
        combine="nested",
        parallel=True,
    )
    data["time"] = new_dates
    return data

In [3]:
ens = [load_data(i, "ens") for i in ["20Z", "21Z", "22Z", "23Z", "00Z", "01Z"]]
ens = [e.assign_coords(lat=e.lat, lon=e.lon, ne=e.ne) for e in ens]

# Make a new DataArray
init = xr.DataArray(data=["20Z", "21Z", "22Z", "23Z", "00Z", "01Z"], dims="init")
init = init.assign_coords({"init": init})

# concat
ens = xr.concat(ens, dim=init).chunk({"init": 1, "time": -1})

# grab UH and rain
rain = ens.rain
uh = ens.uh_0to2

## Other functions

In [6]:
def pmm(da: xr.DataArray) -> xr.DataArray:
    # Create ensemble mean
    ens_mean = da.mean(dim="run").stack(i=("lat", "lon"))

    # Sort
    sorted_mean = ens_mean.sortby(ens_mean)

    # Sort with all ensemble members (no averaging)
    sorted_all = np.sort(da.values.flatten())[:: len(da.run)]

    # Replace values
    sorted_mean.values = sorted_all

    return sorted_mean.unstack("i")